<a href="https://colab.research.google.com/github/dpatinoecheverri/ENV717_S23/blob/main/Assignments/A10/ENV717_A10_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 10
##ENV717- Spring 2023
---

Instructor: [Dalia Patino-Echeverri](mailto:dalia.patino@duke.edu)

Author(s): [Dalia Patino-Echeverri](mailto:dalia.patino@duke.edu), [Mauricio Hernandez](mailto:mmh54@duke.edu), [Abbhijith Hari](mailto:ah580@duke.edu)


[Github repository](https://github.com/dpatinoecheverri/ENV717_S23)


#1.	(25%) 
Develop a Transmission constrained Economic Dispatch Model in Python Pyomo to solve problem 6.6. of the textbook. Submit a knitted HTML file to SAKAI.


<img src="https://drive.google.com/uc?export=view&id=1-00sJ7wEGDg_CoDKqwTxwF91meBIysZY"  width="50%" height="50%">

In [ ]:
# Inporting Libraries
import shutil
import sys
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

!pip install pyomo
!apt-get install -y -qq glpk-utils

from pyomo.environ import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')
#save path
#access path where notebook is stored
path = "/content/gdrive/MyDrive/ENV717-S2023/A10/"
#Change the working directory to the folder
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Read Data

In [ ]:
#Read data
df_gens = pd.read_excel(io="A10_Q1_Data.xlsx", sheet_name="GenData") #read data from generators 
df_loads = pd.read_excel(io="A10_Q1_Data.xlsx", sheet_name="LoadData") #read data from loads tab 
df_lines = pd.read_excel(io="A10_Q1_Data.xlsx", sheet_name="LineData") #read data from line

In [ ]:
#define the size of the sets
num_gens = len(df_gens)
num_nodes = len(df_loads)
num_lines = len(df_lines)

print("We read data for {} power generating units".format(num_gens))
print("We read data for", num_nodes, "nodes")
print("We read data for", num_lines, "transmission lines")

We read data for 4 power generating units
We read data for 3 nodes
We read data for 3 transmission lines


**Define Indexes**

In [ ]:
#Define Indexes

G = np.array([g for g in range(0, num_gens)]) 
N = np.array([n for n in range(0, num_nodes)])
L = np.array([l for l in range(0, num_lines)])

### Organize Network Data: Generators, Lines, and Loads
**Generators Information**

In [ ]:
df_gens.columns

Index(['generator', 'capacity', 'cost_mwh', 'node'], dtype='object')

In [ ]:
##Declare the generator's parameters and assign them the data we read before
cost_mwh = df_gens.loc[:,'cost_mwh'].to_numpy()
pmax = df_gens.loc[:,'capacity'].to_numpy()
gen_node = df_gens.loc[:,'node'].to_numpy()

**Lines Information**

In the data there are three lines defined as: L1-L2, L2-L3, and L3-L1. 

In [ ]:
df_lines.columns

Index(['flow', 'reactance', 'line_capacity', 'from_node', 'to_node'], dtype='object')

In [ ]:
##Declare the transmission lines' parameters and assign them the data we read before
node_from = df_lines.loc[:,'from_node'].to_numpy()
node_to = df_lines.loc[:,'to_node'].to_numpy()
line_react = df_lines.loc[:,'reactance'].to_numpy()
line_capac = df_lines.loc[:,'line_capacity'].to_numpy()

**Loads Information**

In [ ]:
df_loads.columns

Index(['node', 'load'], dtype='object')

In [ ]:
##Declare the loads' parameters and assign them the data read
load_bus = df_loads.loc[:,'node'].to_numpy()
demand = df_loads.loc[:,'load'].to_numpy()

In [ ]:
#Define a function IndicatorMatrix that 
def IndicatorMatrix(nrows, ncols, data_rows):
  '''
  Creates indicator matrix to relate generators with their node and lines with 
  the nodesdataRows is a column vector of NumRows rows. Each element says the 
  column where this row should be a one in the final matrix
  '''
  matrix = np.zeros((nrows, ncols), dtype=int)
  for i in range(0,nrows):
      matrix[i,data_rows[i]-1]=1
  return matrix

**Create Indicator Matrices**

In [ ]:
#Indicator Matrix of G rows and N columns.
#It has 1 in the position g,n if generator g is AT node N, 0 otherwise
mat_gens_in_bus = IndicatorMatrix(num_gens, num_nodes, gen_node)

#Indicator matrix of L rows and N columns. 
# It has 1 in the position l,n if line l departs FROM node N, 0 otherwise
mat_line_from_bus = IndicatorMatrix(num_lines, num_nodes, node_from)

#Indicator matrix of L rows and N columns.
#It has 1 in the position l,n if line l arrives TO node N, 0 otherwise
mat_line_to_bus = IndicatorMatrix(num_lines, num_nodes, node_to)

**Instead of reading the information from Excel, we can could enter all the information directly to the python variables. Not recommended if the network is big.**

In [ ]:
#This information can be stored and read from a file
num_gens = 4
num_nodes = 3
num_lines = 3

cost_mwh = [12, 15, 10, 8]
MaxGenCapac = [150, 200, 150, 400]
#Gens = ['GA', 'GB', 'GC', 'GD']

G = np.array([g for g in range(0,num_gens)]) 
N = np.array([n for n in range(0,num_nodes)])
L = np.array([l for l in range(0,num_lines)])

demand = [400, 80, 40]
line_react = np.array([0.2, 0.3, 0.3])
line_capac = np.array([250, 250, 250])

mat_line_from_bus = np.array([[1,0,0],[0,1,0], [0,0,1]])
mat_line_to_bus = np.array([[0,1,0],[0,0,1], [1,0,0]])
mat_gens_in_bus = np.array([[0,1,0], [1,0,0], [0,0,1], [0,0,1]])

### Defining the Economic Dispatch Model

$$
\begin{equation}
\begin{aligned}
\min_{p_g, f_{n,m}} \quad & \sum_{g=1}^{G}C_gp_g\\
\textrm{s.t.} \quad & \sum_g p_g = \sum_{d}P_d^{D}\\
  &0 \leq p_g \leq P_g^{max} \quad ∀ g \\
  & -F_{n,m} \leq f_{n,m} \leq F_{n,m} \quad \forall n, \forall m \in \Omega_n \\
  &\sum_{m\in Ω_n}X_{n,m}f_{n,m} = 0 \quad \forall n\\
  &\sum_{d \in \Psi_n}P_d^{D} + \sum_{m \in \Omega_n}f_{n,m} - \sum_{g\in \Psi_n}p_g =0 \quad \forall n
\end{aligned}
\end{equation}
$$

In [ ]:
#optimization model definition
def ED_Model():
  m=ConcreteModel()

  #Declaring Parameters
  m.Cost = Set(initialize = cost_mwh)
  m.Gen = Set(initialize = G)
  m.L = Set(initialize = L)
  m.N = Set(initialize = N)
  
  #Create a 'dual' suffix component on the instance, so the solver plugin will 
  #know which suffixes to collect
  m.dual = Suffix(direction = Suffix.IMPORT) 

  #Declaring Decision Variables
  m.p = Var(m.Gen, domain= PositiveReals)
  m.flow = Var(m.L, bounds = (-max(line_capac), max(line_capac))) #This is power flow on a line
  
  #Declaring Constraints
  m.max_gen_constr = Constraint(m.Gen, rule=lambda m, g:  m.p[g] <= pmax[g])
  #Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
  m.kvl_loop_constr = Constraint(rule=lambda m: sum(line_react[l]*m.flow[l] for l in L)==0) 
  m.node_balance_constr = Constraint(m.N, rule=lambda m, n: sum(mat_line_to_bus[l,n]*m.flow[l] for l in L) - 
                                           sum(mat_line_from_bus[l,n]*m.flow[l] for l in L) + 
                                           sum(mat_gens_in_bus[g,n]*m.p[g] for g in G) == demand[n])
  m.max_flow_const=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= line_capac[l])
  m.max_counter_flow_const=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -line_capac[l])

  #Objective Function
  m.system_cost=Objective(expr=sum(m.p[g]*cost_mwh[g] for g in m.Gen), sense=minimize)

  return m


### Solving the Optimization Problem

In [ ]:
m = ED_Model()

SolverFactory('glpk', executable='/usr/bin/glpsol').solve(m).write() # Instead of this line, write "opt.solve(m)" if model's details are not important

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4560.0
  Upper bound: 4560.0
  Number of objectives: 1
  Number of constraints: 15
  Number of variables: 8
  Number of nonzeros: 24
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.016998291015625
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

### Presenting Main Results

In [ ]:
print('SOLUTION')
print('System cost = ${:}'.format(m.system_cost()))
print("--------------------------------------------")
print('Power generated by unit in MW')
for g in G:
  print("Generator {0:2d} = {1:.0f} ".format(g+1, m.p[g]()))
print("--------------------------------------------")
print('Flow on transmission lines in MW')

for l in L:
  print("Line {0:2d} = {1:.0f} ".format(l+1, m.flow[l]()))
print("--------------------------------------------")
print('LMPs in $/MWh')
for n in N:
  print("Node {0:2d} = {1:.2f}".format(n+1, m.dual[m.node_balance_constr[n]]))

SOLUTION
System cost = $4560.0
--------------------------------------------
Power generated by unit in MW
Generator  1 = 80 
Generator  2 = 0 
Generator  3 = 40 
Generator  4 = 400 
--------------------------------------------
Flow on transmission lines in MW
Line  1 = -150 
Line  2 = -150 
Line  3 = 250 
--------------------------------------------
LMPs in $/MWh
Node  1 = 13.33
Node  2 = 12.00
Node  3 = 10.00


In [ ]:
%%shell
jupyter nbconvert --to html "/content/gdrive/MyDrive/ENV717-S2023/A10/ENV717_A10_Solution.ipynb"

[NbConvertApp] Converting notebook /content/gdrive/MyDrive/ENV717-S2023/A10/ENV717_A10_Solution.ipynb to html
[NbConvertApp] Writing 643694 bytes to /content/gdrive/MyDrive/ENV717-S2023/A10/ENV717_A10_Solution.html
